# Update (GEGD)
Using an Area of Interest (AOI) from the `aoi` SQLite Database, queries the Global Enchanced GEOINT Delivery API for data over a given time period, builds the `gegd` table from these results, and then populates the table with these values. Other notebooks should be used for adding additional records to the table.

### Import libraries

In [8]:
# Web Stack
import getpass
import requests

# Data Science stack
import geopandas as gpd

# Custom stack
import sys; sys.path.append("../../")
from database import functions

### User defined variables

In [9]:
dar_id = '19'
start_date = '2023-04-26'
end_date = '2024-04-26'
db = "C:/gis/gaia/data/databases/gaia.db"
connect_id = 'eb33ba21-1782-4ffc-8a5c-4854e21effb9'

### Get Log-in details from user
Hint: !!!

In [10]:
username = input("Enter your username:  ")
password = getpass.getpass("Enter your password:  ")

Enter your username:   1572183293
Enter your password:   ········


### Retreve the Area of Interest record from the `aoi` table corresponding to the supplied DAR number

In [11]:
gdf_aoi = functions.get_aoi(db, dar_id)
gdf_aoi

,dar_id,name,sqkm,geometry
0,19,"SE Atlantic City, NJ",540.23,"POLYGON ((-74.60098 39.40608, -73.74260 39.406..."


### Build Common Query Language (CQL) spatio-temporal filter

In [12]:
spatial_filter = "INTERSECTS(geometry, {})".format(gdf_aoi['geometry'][0].wkt)
temporal_filter = "acquisitionDate > {} AND acquisitionDate < {}".format(start_date, end_date) 
cql_filter = ' AND '.join([spatial_filter, temporal_filter])
cql_filter

'INTERSECTS(geometry, POLYGON ((-74.600977 39.406081, -73.7426 39.406081, -73.745663 38.739391, -74.597914 38.739391, -74.600977 39.406081))) AND acquisitionDate > 2023-04-26 AND acquisitionDate < 2024-04-26'

### Search GEGD

In [13]:
url = 'https://evwhs.digitalglobe.com/catalogservice/wfsaccess'

params = {
    'service': 'WFS',
    'version': '2.0.0',
    'request': 'GetFeature',
    'typeName': 'DigitalGlobe:FinishedFeature',
    'outputFormat': 'application/json',
    'connectId': connect_id,
    'cql_filter': cql_filter
}

results = requests.get(url=url, params=params, auth=(username, password))
r = results.json()
r

{'type': 'FeatureCollection',
 'totalFeatures': 8,
 'features': [{'type': 'Feature',
   'id': '2a16e31c01528687d7f9d67c987f0697',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[38.81524162, -74.00414475],
      [38.81613275, -73.9724081],
      [38.81702039, -73.94067136],
      [38.81790457, -73.90893451],
      [38.81878531, -73.87719757],
      [38.81966262, -73.84546053],
      [38.82053653, -73.81372338],
      [38.82140706, -73.78198614],
      [38.82227422, -73.75024881],
      [38.82313805, -73.71851137],
      [38.82399855, -73.68677383],
      [38.82485574, -73.65503619],
      [38.82570966, -73.62329846],
      [38.82656031, -73.59156062],
      [38.82740771, -73.55982269],
      [38.82825189, -73.52808465],
      [38.82909285, -73.49634651],
      [38.82993063, -73.46460828],
      [38.83076524, -73.43286994],
      [38.8315967, -73.4011315],
      [38.83242502, -73.36939296],
      [38.83325023, -73.33765433],
      [38.83407233, -73.30591559],
      [38.83489136

### Create a GeoDataFrame from query results

In [14]:
gdf = functions.gdf_from_gegd(r, dar_id)
gdf.head()

,id,dar_id,legacy_id,factory_order_number,acquisition_date,source,source_unit,product_type,cloud_cover,off_nadir_angle,...,asset_name,per_pixel_x,per_pixel_y,crs_from_pixels,age_days,ingest_date,company_name,copyright,niirs,geometry
0,2a16e31c01528687d7f9d67c987f0697,19,10200100EB3A3C00,200004852490,2024-03-20 09:19:45,WV01,Strip,Panchromatic,0.322,4.200061,...,FINISHED,0.508811,-0.508811,EPSG:32737,54.0,2024-03-20 12:55:44,DigitalGlobe,Image Copyright 2024 DigitalGlobe Inc,5.0,"POLYGON ((-74.00414 38.81524, -73.97241 38.816..."
1,50034342f917df2dd9a647eb8cd4d15c,19,10200100E9705D00,200004781866,2024-03-03 09:17:07,WV01,Strip,Panchromatic,0.000,4.507185,...,FINISHED,0.509539,-0.509539,EPSG:32737,71.0,2024-03-03 10:50:39,DigitalGlobe,Image Copyright 2024 DigitalGlobe Inc,5.0,"POLYGON ((-73.99980 38.66231, -73.96823 38.663..."
2,e9bc54e26d4a559c371ac23e422d4cdc,19,1040010093B0F100,200004763798,2024-03-01 06:05:05,WV03_VNIR,Strip,Pan Sharpened Natural Color,0.000,18.684578,...,FINISHED,0.320210,-0.320210,EPSG:32737,73.0,2024-03-01 11:04:00,DigitalGlobe,Image Copyright 2024 DigitalGlobe Inc,5.0,"POLYGON ((-74.01397 38.58808, -73.98199 38.588..."
3,bccc8d38ab57195f14a97b17fd94afea,19,104001009276BD00,200004763806,2024-03-01 06:04:43,WV03_VNIR,Strip,Pan Sharpened Natural Color,0.000,21.629143,...,FINISHED,0.349728,-0.349728,EPSG:32737,73.0,2024-03-01 10:40:58,DigitalGlobe,Image Copyright 2024 DigitalGlobe Inc,5.0,"POLYGON ((-74.01652 38.93963, -73.98436 38.939..."
4,4f72691d54f374e04274ce8bd06d62cd,19,10200100E5543F00,016128198-10,2023-12-04 09:21:10,WV01,Strip,Panchromatic,0.000,17.665314,...,FINISHED,0.500000,-0.500000,EPSG:32737,161.0,2023-12-04 13:17:09,DigitalGlobe,Image Copyright 2023 DigitalGlobe Inc,4.8,"POLYGON ((-74.01441 38.26405, -73.98258 38.265..."


### Insert GEGD IDs into `gegd` table

In [15]:
functions.insert_pk(db, 'gegd', gdf)

Done inserting Imagery IDs into table!


### Update `gegd` records from GEGD results

In [16]:
columns = gdf.columns[1:]
for i, row in gdf.iterrows():
    iid = row['id']
    print("Updating information for Imagery ID: {}".format(iid))
    for column in columns:
        functions.database_activity(db, 'gegd', iid, column, row[column])

Updating information for Imagery ID: 2a16e31c01528687d7f9d67c987f0697
Updating information for Imagery ID: 50034342f917df2dd9a647eb8cd4d15c
Updating information for Imagery ID: e9bc54e26d4a559c371ac23e422d4cdc
Updating information for Imagery ID: bccc8d38ab57195f14a97b17fd94afea
Updating information for Imagery ID: 4f72691d54f374e04274ce8bd06d62cd
Updating information for Imagery ID: 9a81b2eadc1ce3f00eb0808882b8fd40
Updating information for Imagery ID: cca5a3403b4e21357229e17589b0c81e
Updating information for Imagery ID: df61facab4171289a30adbaab1e5f08b


### Select newly inserted AOIs, make a GeoDataFrame for validation

In [17]:
gdf = functions.validate_updates(db, 'gegd', gdf, dar_id)
# Note that the GDF shape matches that from the above
print(gdf.shape)
gdf.head()

(8, 26)


,id,dar_id,legacy_id,factory_order_number,acquisition_date,source,source_unit,product_type,cloud_cover,off_nadir_angle,...,asset_name,per_pixel_x,per_pixel_y,crs_from_pixels,age_days,ingest_date,company_name,copyright,niirs,geometry
0,2a16e31c01528687d7f9d67c987f0697,19,10200100EB3A3C00,200004852490,2024-03-20 09:19:45,WV01,Strip,Panchromatic,0.322,4.200061,...,FINISHED,0.508811,-0.508811,EPSG:32737,54,2024-03-20 12:55:44,DigitalGlobe,Image Copyright 2024 DigitalGlobe Inc,5.0,"POLYGON ((-74.00414 38.81524, -73.97241 38.816..."
1,50034342f917df2dd9a647eb8cd4d15c,19,10200100E9705D00,200004781866,2024-03-03 09:17:07,WV01,Strip,Panchromatic,0.000,4.507185,...,FINISHED,0.509539,-0.509539,EPSG:32737,71,2024-03-03 10:50:39,DigitalGlobe,Image Copyright 2024 DigitalGlobe Inc,5.0,"POLYGON ((-73.99980 38.66231, -73.96823 38.663..."
2,e9bc54e26d4a559c371ac23e422d4cdc,19,1040010093B0F100,200004763798,2024-03-01 06:05:05,WV03_VNIR,Strip,Pan Sharpened Natural Color,0.000,18.684578,...,FINISHED,0.320210,-0.320210,EPSG:32737,73,2024-03-01 11:04:00,DigitalGlobe,Image Copyright 2024 DigitalGlobe Inc,5.0,"POLYGON ((-74.01397 38.58808, -73.98199 38.588..."
3,bccc8d38ab57195f14a97b17fd94afea,19,104001009276BD00,200004763806,2024-03-01 06:04:43,WV03_VNIR,Strip,Pan Sharpened Natural Color,0.000,21.629143,...,FINISHED,0.349728,-0.349728,EPSG:32737,73,2024-03-01 10:40:58,DigitalGlobe,Image Copyright 2024 DigitalGlobe Inc,5.0,"POLYGON ((-74.01652 38.93963, -73.98436 38.939..."
4,4f72691d54f374e04274ce8bd06d62cd,19,10200100E5543F00,016128198-10,2023-12-04 09:21:10,WV01,Strip,Panchromatic,0.000,17.665314,...,FINISHED,0.500000,-0.500000,EPSG:32737,161,2023-12-04 13:17:09,DigitalGlobe,Image Copyright 2023 DigitalGlobe Inc,4.8,"POLYGON ((-74.01441 38.26405, -73.98258 38.265..."


### Plot Areas of Interest on an Interactive Map

In [18]:
mp = functions.quick_map(gdf, gdf_aoi['geometry'].to_json())
mp

# End